WARNING: this example is not yet fully functional

In [1]:
from line_solver import *
GlobalConstants.setVerbose(VerboseLevel.STD)

A basic M/M/1 with explicit definition of a classwitch node
Recommended ClassSwitch declaration style

In [2]:
model = Network('mm1cs')

# Block 1: nodes
node = np.empty(4, dtype=object)
node[0] = Source(model, 'Source 1')
node[1] = Queue(model, 'Queue 1', SchedStrategy.FCFS)
node[2] = Sink(model, 'Sink 1')
node[3] = ClassSwitch(model, 'ClassSwitch 1')

# Block 2: classes
jobclass = np.empty(2, dtype=object)
jobclass[0] = OpenClass(model, 'Class1', 0)
jobclass[1] = OpenClass(model, 'Class2', 0)

node[0].setArrival(jobclass[0], Exp.fitMean(10.00)) # (Source 1,Class1)
node[0].setArrival(jobclass[1], Exp.fitMean(2.00)) # (Source 1,Class2)

node[1].setService(jobclass[0], Exp.fitMean(1.00)) # (Queue 1,Class1)
node[1].setService(jobclass[1], Exp.fitMean(1.00)) # (Queue 1,Class2)


In [3]:
# Block 3: topology
# The class switching matrix can now be declared after the classes, so the
# ClassSwitch node can be declared outside Block 1.

csmatrix = node[3].initClassSwitchMatrix() # element (i,j) = probability that class i switches to j
csmatrix[jobclass[0]][jobclass[0]] = 0.3
csmatrix[jobclass[0]][jobclass[1]] = 0.7
csmatrix[jobclass[1]][jobclass[0]] = 1.0
csmatrix[jobclass[1]][jobclass[1]] = 0.0
node[3].setClassSwitchingMatrix(csmatrix)

P = model.initRoutingMatrix() # initialize routing matrix
P.set(jobclass[0],jobclass[0],node[0],node[3],1.0) # (Source 1,Class1) -> (ClassSwitch 1,Class1)
P.set(jobclass[0],jobclass[0],node[1],node[2],1.0) # (Queue 1,Class1) -> (Sink 1,Class1)
P.set(jobclass[0],jobclass[0],node[3],node[1],1.0) # (ClassSwitch 1,Class1) -> (Queue 1,Class1)
P.set(jobclass[1],jobclass[1],node[0],node[3],1.0) # (Source 1,Class2) -> (ClassSwitch 1,Class2)
P.set(jobclass[1],jobclass[1],node[1],node[2],1.0) # (Queue 1,Class2) -> (Sink 1,Class2)
P.set(jobclass[1],jobclass[1],node[3],node[1],1.0) # (ClassSwitch 1,Class2) -> (Queue 1,Class2)
model.link(P)

model.printRoutingMatrix()

Source 1 [Class1] => ClassSwitch 1 [Class1] : Pr=1.0
Source 1 [Class2] => ClassSwitch 1 [Class2] : Pr=1.0
Queue 1 [Class1] => Sink 1 [Class1] : Pr=1.0
Queue 1 [Class2] => Sink 1 [Class2] : Pr=1.0
Sink 1 [Class1] => Source 1 [Class1] : Pr=0.16666666666666669
Sink 1 [Class1] => Source 1 [Class2] : Pr=0.8333333333333334
Sink 1 [Class2] => Source 1 [Class1] : Pr=0.16666666666666669
Sink 1 [Class2] => Source 1 [Class2] : Pr=0.8333333333333334
ClassSwitch 1 [Class1] => Queue 1 [Class1] : Pr=0.3
ClassSwitch 1 [Class1] => Queue 1 [Class2] : Pr=0.7
ClassSwitch 1 [Class2] => Queue 1 [Class1] : Pr=1.0


In [4]:
# TODO: getAvgChainTable not yet available in JLINE
AvgTable = SolverMVA(model).getAvgTable()

    Station JobClass      QLen  Util     RespT    ResidT  ArvR  Tput
0  Source 1   Class1  0.000000  0.00  0.000000  0.000000  0.10  0.10
1  Source 1   Class2  0.000000  0.00  0.000000  0.000000  0.50  0.50
2   Queue 1   Class1  1.324859  0.53  2.499734  2.208099  0.53  0.53
3   Queue 1   Class2  0.174981  0.07  2.499734  0.291636  0.07  0.07


In [5]:
AvgSysTable = SolverMVA(model).getAvgSysTable()

#AvgTable = SolverJMT(model).getAvgChainTable()


    Chain       JobClasses  SysRespT  SysTput
0  Chain0  (Class1 Class2)  2.499734      0.6
